In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import pickle

In [2]:
df1 = pd.read_csv('tmdb_5000_movies.csv')
df2 = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies = df1.merge(df2, on = 'title')

In [4]:
movies.shape

(4809, 23)

In [5]:
#Function to convert required data from dictionary into list
def tolist(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [6]:
#Function to convert first 3 actor names data from dictionary into list
def tolist1(obj):
    l = []
    c = 0
    for i in ast.literal_eval(obj):
        if c<=2:
            l.append(i['name'])
            c+=1
        else:
            break
    return l

In [7]:
movies['cast'] = movies['cast'].apply(tolist1)

In [8]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [9]:
#Function to director name dictionary into list
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
        
    return l

In [10]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [11]:
movies.dropna(inplace=True)

In [12]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [13]:
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])

In [14]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [15]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])


In [16]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))


In [17]:
new['tags'] = new['tags'].apply(lambda x:x.lower())

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words= 'english')

In [19]:
vector = cv.fit_transform(new['tags']).toarray()

In [20]:
vector.shape


(4806, 5000)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity


In [22]:
similarity = cosine_similarity(vector)


In [23]:
similarity[0]

array([1., 0., 0., ..., 0., 0., 0.])

In [24]:
movie_index = new.index[new['title'] == 'Spectre'].tolist()[0]

In [25]:
def recommend_movie(movie):
    #index in new dataframe of selected movie
    movie_index = new.index[new['title'] == movie].tolist()[0]
    distances = similarity[movie_index]
    #create a numbered list of distances of selecte movies and sort,
    #which returns next top 5 values(excluding first as it is the same movie)
    movies_list = sorted(list(enumerate(distances)),reverse = True, key = lambda x:x[1] )[1:6]
    
    
    for i in movies_list:
        print(new.iloc[i[0]].title)
    

In [26]:
sorted(list(enumerate(similarity[7])),reverse = True, key = lambda x:x[1] )[0:5]

[(7, 0.9999999999999998),
 (68, 0.18257418583505536),
 (16, 0.17568209223157663),
 (531, 0.14808721943977307),
 (31, 0.14638501094227996)]

In [27]:
recommend_movie('Avatar')

Apollo 18
Beowulf
The Book of Life
Tears of the Sun
Aliens


In [28]:
#pickle.dump(new,open('movie_list.pkl','wb'))
#pickle.dump(similarity,open('similarity.pkl','wb'))
#pickle.dump(new.to_dict(), open('movie_dict.pkl', 'wb'))